In [1]:
import os
import pandas as pd
import json
import os, sys
print(os.path.join(os.path.dirname(os.getcwd()), "app"))

sys.path.append("/Users/officialbiznas/Documents/GitHub/stockFries/app")
import scraper
from config import fund_dict
import requests
from bs4 import BeautifulSoup

/Users/officialbiznas/Documents/GitHub/stockFries/app


## Testing Each Function

In [4]:
def forms_scraper(fund_link):
    """Takes the SEC fund link to return the 2 most recent filing URLS"""
    r = requests.get(fund_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    filing_links = []
    table = soup.find_all('table')[2]
    rows = [i for i in table.find_all('tr')]
    for i in rows:
        cols = i.find('td')
        if cols is not None:
            if '13F-HR' in cols.text:
                base_link = 'https://www.sec.gov'
                link = i.find('a').get('href')
                filing_links.append(base_link + link)

    form_links = []
    for i in filing_links[:2]:
        r = requests.get(i)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table')
        rows = [i for i in table.find_all('tr')]
        form_links.append(base_link + rows[3].find('a').get('href'))
    return form_links


def shares_scraper(form_link):
    """Takes a filing link and appends all holdings to a list"""
    r = requests.get(form_link)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('table')[3]
    rows = table.find_all('tr')[3:]

    shares_list = []
    rows_text = []
    for i in rows:
        cols = i.find_all('td')
        cols_text = []
        for col in cols:
            if col is not None:
                cols_text.append(col.text)
        rows_text.append(cols_text)

    for i in rows_text:
        if (i[6] != 'Put') or (i[6] != 'Call'):
            shares_list.append({'CUSIP': i[2][:9], 'Company': i[0], 'Value': int(i[3].replace(',', '')),
                                'Shares': int(i[4].replace(',', ''))})

    return shares_list
def pandas_analysis_test(fund_dict):
    """Takes the fund dict and gets the filings URLS from forms_scraper, then extends two aggregate lists for the current and previous holdings across all funds before analysis in pandas and export to json txt"""
    list_current = []
    list_previous = []
    for k, v in fund_dict.items():
        try:
            forms = forms_scraper(v)
            print(forms)
            shares_list_current = shares_scraper(forms[0])
            list_current.extend(shares_list_current)
            shares_list_previous = shares_scraper(forms[1])
            list_previous.extend(shares_list_previous)
        except:
            pass

    df_current = pd.DataFrame(list_current)
    dfc = df_current.groupby('CUSIP').agg(
        {'Company': 'first', 'Value': 'sum', 'Shares': 'sum'})
    dfc['Company'] = dfc['Company'].str.title()
    dfc['Ownership'] = (dfc['Value'] / dfc['Value'].sum()) * 100
    dfc = dfc.sort_values('Ownership', ascending=False).round(2)

    df_previous = pd.DataFrame(list_previous)
    dfp = df_previous.groupby('CUSIP').agg(
        {'Company': 'first', 'Value': 'sum', 'Shares': 'sum'})
    dfp['Company'] = dfp['Company'].str.title()

    dff = pd.merge(dfc, dfp, on=['CUSIP', 'Company'], how='outer')
    dff['Change'] = ((dff['Shares_x'] - dff['Shares_y']) /
                     dff['Shares_x']) * 100

    json_df = dff.round(2).to_json(orient='split')
    #with open('os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt'), 'w') as f:
    #    json.dump(json_df, f)
    return dff

In [12]:
fund_link_test = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=1535392&owner=exclude&count=40&hidefilings=0'
form_links_test = forms_scraper(fund_link_test)

In [22]:
shares_list_test = shares_scraper(form_links_test[0])
print(shares_list_test)

[{'CUSIP': '03076C106', 'Company': 'AMERIPRISE FINL INC', 'Value': 6497, 'Shares': 43301}, {'CUSIP': '04878Q863', 'Company': 'ATLANTIC POWER CORP', 'Value': 4405, 'Shares': 2202685}, {'CUSIP': 'G0772R208', 'Company': 'BANK OF NT BUTTERFIELD&SON L', 'Value': 1508, 'Shares': 61828}, {'CUSIP': '07987C204', 'Company': 'BELLUS HEALTH INC NEW', 'Value': 2138, 'Shares': 207778}, {'CUSIP': '09609G100', 'Company': 'BLUEBIRD BIO INC', 'Value': 7912, 'Shares': 129622}, {'CUSIP': '097793400', 'Company': 'BONANZA CREEK ENERGY INC', 'Value': 30540, 'Shares': 2060700}, {'CUSIP': '10948W103', 'Company': 'BRIGHTSPHERE INVT GROUP INC', 'Value': 28124, 'Shares': 2257173}, {'CUSIP': '110122157', 'Company': 'BRISTOL-MYERS SQUIBB CO', 'Value': 14127, 'Shares': 3946145}, {'CUSIP': '12477X106', 'Company': 'CAI INTERNATIONAL INC', 'Value': 14086, 'Shares': 845512}, {'CUSIP': '14888U101', 'Company': 'CATALYST PHARMACEUTICALS INC', 'Value': 14462, 'Shares': 3130195}, {'CUSIP': '16383L106', 'Company': 'CHEMOCENTR

In [6]:
%%time
df_test = pandas_analysis_test(fund_dict)

['https://www.sec.gov/Archives/edgar/data/1535392/000156761920015183/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1535392/000156761920010282/xslForm13F_X01/form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1544676/000154467620000008/xslForm13F_X01/13f.xml', 'https://www.sec.gov/Archives/edgar/data/1544676/000154467620000006/xslForm13F_X01/13fv2.xml']
['https://www.sec.gov/Archives/edgar/data/1425649/000110465920095369/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1425649/000110465920062428/xslForm13F_X01/infotable.xml']
[]
['https://www.sec.gov/Archives/edgar/data/1104186/000110418620000005/xslForm13F_X01/t13f-0620.xml', 'https://www.sec.gov/Archives/edgar/data/1104186/000110418620000004/xslForm13F_X01/t13f-0320.xml']
['https://www.sec.gov/Archives/edgar/data/1226355/000122635519000012/xslForm13F_X01/Harvest13f_3q2019.inftab.xml', 'https://www.sec.gov/Archives/edgar/data/1226355/000122635519000009/xslForm13F_X01

['https://www.sec.gov/Archives/edgar/data/919468/000117266119001064/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/919468/000117266119000232/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1517137/000092189520002245/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1517137/000092189520001444/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1647251/000164725120000006/xslForm13F_X01/Form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1647251/000164725120000004/xslForm13F_X01/Form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1167483/000091957420005337/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1167483/000091957420003646/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1345471/000134547120000054/xslForm13F_X01/infochart13F0630amend2.xml', 'https://www.sec.gov/Archives/edgar/data/1345471/000134547120000044/xslForm13F_

In [7]:
df_test.tail()

,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change
CUSIP,,,,,,,
N6596X109,Nxp Semiconductors Nv,NaN,NaN,NaN,2128.0,25663.0,NaN
N70544106,Playa Hotels & Resorts Nv,NaN,NaN,NaN,4103.0,2344296.0,NaN
Y2573F102,Flex Ltd,NaN,NaN,NaN,98359.0,11744352.0,NaN
Y41053102,International Seaways Inc,NaN,NaN,NaN,21923.0,917646.0,NaN
Y75638109,Atlas Corp,NaN,NaN,NaN,1423.0,185009.0,NaN


## Tetsting SRC Code

In [2]:
%%time
scraper.pandas_analysis(fund_dict)

['https://www.sec.gov/Archives/edgar/data/1535392/000156761920015183/xslForm13F_X01/form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1535392/000156761920010282/xslForm13F_X01/form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1544676/000154467620000008/xslForm13F_X01/13f.xml', 'https://www.sec.gov/Archives/edgar/data/1544676/000154467620000006/xslForm13F_X01/13fv2.xml']
['https://www.sec.gov/Archives/edgar/data/1425649/000110465920095369/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1425649/000110465920062428/xslForm13F_X01/infotable.xml']
[]
['https://www.sec.gov/Archives/edgar/data/1104186/000110418620000005/xslForm13F_X01/t13f-0620.xml', 'https://www.sec.gov/Archives/edgar/data/1104186/000110418620000004/xslForm13F_X01/t13f-0320.xml']
['https://www.sec.gov/Archives/edgar/data/1226355/000122635519000012/xslForm13F_X01/Harvest13f_3q2019.inftab.xml', 'https://www.sec.gov/Archives/edgar/data/1226355/000122635519000009/xslForm13F_X01

['https://www.sec.gov/Archives/edgar/data/919468/000117266119001064/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/919468/000117266119000232/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1517137/000092189520002245/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1517137/000092189520001444/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1647251/000164725120000006/xslForm13F_X01/Form13fInfoTable.xml', 'https://www.sec.gov/Archives/edgar/data/1647251/000164725120000004/xslForm13F_X01/Form13fInfoTable.xml']
['https://www.sec.gov/Archives/edgar/data/1167483/000091957420005337/xslForm13F_X01/infotable.xml', 'https://www.sec.gov/Archives/edgar/data/1167483/000091957420003646/xslForm13F_X01/infotable.xml']
['https://www.sec.gov/Archives/edgar/data/1345471/000134547120000054/xslForm13F_X01/infochart13F0630amend2.xml', 'https://www.sec.gov/Archives/edgar/data/1345471/000134547120000044/xslForm13F_

In [3]:
fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
print(fn)
with open(fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split').reset_index()
df.tail()

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt


,index,Company,Value_x,Shares_x,Ownership,Value_y,Shares_y,Change
2612,N6596X109,Nxp Semiconductors Nv,NaN,NaN,NaN,2128.0,25663.0,NaN
2613,N70544106,Playa Hotels & Resorts Nv,NaN,NaN,NaN,4103.0,2344296.0,NaN
2614,Y2573F102,Flex Ltd,NaN,NaN,NaN,98359.0,11744352.0,NaN
2615,Y41053102,International Seaways Inc,NaN,NaN,NaN,21923.0,917646.0,NaN
2616,Y75638109,Atlas Corp,NaN,NaN,NaN,1423.0,185009.0,NaN


In [2]:
fn = os.path.join(os.path.dirname(os.getcwd()), 'app', 'json_df.txt') #app/json_df.txt
print(fn)
with open(fn, 'r') as f:
        json_df = json.load(f)
df = pd.read_json(json_df, orient='split').reset_index()
df.tail()

/Users/officialbiznas/Documents/GitHub/stockFries/app/json_df.txt


,index,Company,Shares_x,Value_x,Ownership,Shares_y,Value_y,Change
2256,N70544,Playa Hotels & Resorts Nv,NaN,NaN,NaN,2344296.0,4103.0,NaN
2257,N72482,Qiagen Nv,NaN,NaN,NaN,54200000.0,63232.0,NaN
2258,Y2573F,Flex Ltd,NaN,NaN,NaN,11744352.0,98359.0,NaN
2259,Y41053,International Seaways Inc,NaN,NaN,NaN,917646.0,21923.0,NaN
2260,Y75638,Atlas Corp,NaN,NaN,NaN,185009.0,1423.0,NaN


In [17]:
df.head()

,index,Company,Shares_x,Value_x,Ownership,Shares_y,Value_y,Change
0,88160R,Tesla Inc,42116277.0,192177977.0,2.94,NaN,NaN,NaN
1,097023,Boeing Co,3658828.0,167087119.0,2.56,394102.0,58777.0,89.23
2,64110L,Netflix Inc,5576767.0,151505143.0,2.32,9129848.0,3428260.0,-63.71
3,037833,Apple Inc,375380.0,136938624.0,2.09,347481.0,88361.0,7.43
4,30231G,Exxon Mobil Corp,3060000.0,136843200.0,2.09,3707932.0,140056.0,-21.17
